In [14]:
import hnswlib
import numpy as np
import networkx as nx
import time

In [2]:
def parse_nodes(path):
    contents = open(path).read().splitlines()
    nodes = [[int(x) for x in line.split()] for line in contents]
    nodes = [ set(x) for x in nodes ]
    return nodes

In [3]:
def load_soln(path): # iterate each line as an edge
    parseNode = lambda x: tuple(sorted([int(y) for y in x.split(' ')]))
    graph = nx.read_edgelist(path, create_using=nx.DiGraph(), nodetype=parseNode, delimiter=' -> ')

    print(list(graph.edges())[:500])
    print(f"There are {len(graph.nodes())} nodes and {len(graph.edges())} edges")
    
    return graph

# Building Graph via Transitive Reduction

In [4]:
nodes = parse_nodes('../../data/example/13.txt')
nodes = [tuple(sorted(x)) for x in nodes]
graph = nx.DiGraph()
graph.add_nodes_from(nodes)

# adding edges from every node to every other node
for i in range(len(nodes)):
    for j in range(i+1, len(nodes)):
        if (set(nodes[i]).issubset(set(nodes[j]))):
            graph.add_edge(nodes[i], nodes[j])

tx = nx.transitive_reduction(graph)
tx.edges


OutEdgeView([((2, 5, 570), (1, 2, 4, 5, 11, 121, 468, 570)), ((2, 5, 570), (1, 2, 4, 5, 7, 11, 468, 570)), ((1, 5), (1, 3, 5, 189)), ((1, 5), (1, 3, 5, 8, 477)), ((1, 5), (1, 5, 11)), ((1, 5), (1, 5, 6)), ((1, 5), (1, 5, 468, 570)), ((1, 5, 11), (1, 2, 4, 5, 11, 121, 468, 570)), ((1, 5, 11), (1, 2, 4, 5, 7, 11, 468, 570)), ((1, 3, 5, 189), (1, 3, 5, 189, 669)), ((1, 3, 5, 189), (1, 3, 5, 189, 204, 477)), ((1, 3, 5, 189, 204, 477), (1, 3, 5, 189, 204, 477, 669)), ((1, 3, 5, 189, 669), (1, 3, 5, 189, 204, 477, 669)), ((1, 3, 5, 189, 204, 477, 669), (1, 3, 5, 189, 204, 477, 633, 669)), ((1, 5, 468, 570), (1, 2, 4, 5, 7, 11, 468, 570))])

In [5]:
soln = load_soln('../../data/soln/13.txt')

[((2, 5, 570), (1, 2, 4, 5, 11, 121, 468, 570)), ((2, 5, 570), (1, 2, 4, 5, 7, 11, 468, 570)), ((1, 5), (1, 5, 11)), ((1, 5), (1, 3, 5, 189)), ((1, 5), (1, 5, 468, 570)), ((1, 5), (1, 5, 6)), ((1, 5), (1, 3, 5, 8, 477)), ((1, 5, 11), (1, 2, 4, 5, 11, 121, 468, 570)), ((1, 5, 11), (1, 2, 4, 5, 7, 11, 468, 570)), ((1, 3, 5, 189), (1, 3, 5, 189, 204, 477)), ((1, 3, 5, 189), (1, 3, 5, 189, 669)), ((1, 5, 468, 570), (1, 2, 4, 5, 11, 121, 468, 570)), ((1, 5, 468, 570), (1, 2, 4, 5, 7, 11, 468, 570)), ((1, 3, 5, 189, 204, 477), (1, 3, 5, 189, 204, 477, 669)), ((1, 3, 5, 189, 669), (1, 3, 5, 189, 204, 477, 669)), ((1, 3, 5, 189, 204, 477, 669), (1, 3, 5, 189, 204, 477, 633, 669))]
There are 13 nodes and 16 edges


In [6]:
for node in soln.nodes:
    out_soln = soln.out_edges(node)
    out_graph = graph.out_edges(node)
    diff = set(out_soln) - set(out_graph)
    print('TRUE', out_soln, 'INNOV', out_graph, 'DIFF', diff)
    print(len(diff))

TRUE [((2, 5, 570), (1, 2, 4, 5, 11, 121, 468, 570)), ((2, 5, 570), (1, 2, 4, 5, 7, 11, 468, 570))] INNOV [((2, 5, 570), (1, 2, 4, 5, 11, 121, 468, 570)), ((2, 5, 570), (1, 2, 4, 5, 7, 11, 468, 570))] DIFF set()
0
TRUE [] INNOV [] DIFF set()
0
TRUE [] INNOV [] DIFF set()
0
TRUE [((1, 5), (1, 5, 11)), ((1, 5), (1, 3, 5, 189)), ((1, 5), (1, 5, 468, 570)), ((1, 5), (1, 5, 6)), ((1, 5), (1, 3, 5, 8, 477))] INNOV [((1, 5), (1, 5, 11)), ((1, 5), (1, 3, 5, 189)), ((1, 5), (1, 3, 5, 189, 204, 477)), ((1, 5), (1, 3, 5, 189, 669)), ((1, 5), (1, 3, 5, 189, 204, 477, 669)), ((1, 5), (1, 3, 5, 189, 204, 477, 633, 669)), ((1, 5), (1, 2, 4, 5, 11, 121, 468, 570)), ((1, 5), (1, 5, 468, 570)), ((1, 5), (1, 5, 6)), ((1, 5), (1, 2, 4, 5, 7, 11, 468, 570)), ((1, 5), (1, 3, 5, 8, 477))] DIFF set()
0
TRUE [((1, 5, 11), (1, 2, 4, 5, 11, 121, 468, 570)), ((1, 5, 11), (1, 2, 4, 5, 7, 11, 468, 570))] INNOV [((1, 5, 11), (1, 2, 4, 5, 11, 121, 468, 570)), ((1, 5, 11), (1, 2, 4, 5, 7, 11, 468, 570))] DIFF set()
0


## Results 

does **NOT** work as an algorithm. In this experiment a graph is structed with each node having connections to every other node
it is the superset of. Then, a transitive reduction is applied that removes most of the edges. In general, while often close
to the real solution, the transitive reduction is slightly off. It is not a supplement to the standard algorithm.